#  Variational Inference using ADVI

Variational inference is a method for approximating complex Bayesian posterior distributions using simpler, parameterized distributions.
The Automatic Differentiation Variational Inference (ADVI) algorithm
searches over a family of simple densities to find the best
approximate posterior density.
ADVI produces an estimate of the parameter means together with a sample
from the approximate posterior density.

ADVI uses stochastic gradient ascent to approximate the variational objective function, the evidence lower bound or ELBO.
The algorithm ascends these gradients using an adaptive stepsize sequence
that has one parameter ``eta`` which is adjusted during warmup.
The number of draws used to approximate the ELBO is denoted by ``elbo_samples``. 
ADVI heuristically determines a rolling window over which it computes
the average and the median change of the ELBO.
When this change falls below a threshold, denoted by ``tol_rel_obj``,
the algorithm is considered to have converged.

### Example: variational inference for model ``bernoulli.stan``

The [CmdStanModel variational](https://mc-stan.org/cmdstanpy/api.html#cmdstanpy.CmdStanModel.variational) method
wraps the CmdStan [variational](https://mc-stan.org/docs/cmdstan-guide/variational-config.html) method.

In [1]:
import os
from cmdstanpy.model import CmdStanModel
from cmdstanpy.utils import cmdstan_path

bernoulli_dir = os.path.join(cmdstan_path(), 'examples', 'bernoulli')
stan_file = os.path.join(bernoulli_dir, 'bernoulli.stan')
data_file = os.path.join(bernoulli_dir, 'bernoulli.data.json')
# instantiate, compile bernoulli model
model = CmdStanModel(stan_file=stan_file)
# run CmdStan's variational inference method, returns object `CmdStanVB`
vi = model.variational(data=data_file)

14:28:58 - cmdstanpy - INFO - Chain [1] start processing


14:28:58 - cmdstanpy - INFO - Chain [1] done processing


The class [`CmdStanVB`](https://mc-stan.org/cmdstanpy/api.html#cmdstanvb) provides the following properties to access information about the parameter names, estimated means, and the sample:

  + `column_names` - list of column names
  + `columns` - number of columns
  + `eta` - step size scaling parameter
  + `variational_params_dict` - inferred parameter means as a Dict.
  + `variational_params_np` - inferred parameter means as a numpy.ndarray.
  + `variational_params_pd` - inferred parameter means as a pandas.DataFrame.
  + `variational_sample` - the set of approximate posterior output draws ad a numpy.ndarray.
  + `variational_sample_pd` - the set of approximate posterior output draws ad a pandas.DataFrame.

In [2]:
print(vi.column_names)

('lp__', 'log_p__', 'log_g__', 'theta')


In [3]:
print(vi.variational_params_dict['theta'])

0.232085


In [4]:
print(vi.variational_sample.shape)

(1000, 4)


These estimates are only valid if the algorithm has converged to a good
approximation. When the algorithm fails to do so, the `variational`
method will throw a `RuntimeError`.


In [5]:
model_fail = CmdStanModel(stan_file='eta_should_fail.stan')
vi_fail = model_fail.variational()

14:28:58 - cmdstanpy - INFO - compiling stan file /home/runner/work/cmdstanpy/cmdstanpy/docsrc/users-guide/examples/eta_should_fail.stan to exe file /home/runner/work/cmdstanpy/cmdstanpy/docsrc/users-guide/examples/eta_should_fail


14:29:07 - cmdstanpy - INFO - compiled model executable: /home/runner/work/cmdstanpy/cmdstanpy/docsrc/users-guide/examples/eta_should_fail


14:29:07 - cmdstanpy - INFO - Chain [1] start processing


14:29:07 - cmdstanpy - INFO - Chain [1] done processing


RuntimeError: The algorithm may not have converged.
If you would like to inspect the output, re-call with require_converged=False

Unless you set `require_converged=False`:

In [6]:
vi_fail = model_fail.variational(require_converged=False)

14:29:08 - cmdstanpy - INFO - Chain [1] start processing


14:29:08 - cmdstanpy - INFO - Chain [1] done processing


14:29:08 - cmdstanpy - WARNING - The algorithm may not have converged.
Proceeding because require_converged is set to False


This lets you inspect the output to try to diagnose the issue with the model

In [7]:
vi_fail.variational_params_dict

OrderedDict([('lp__', np.float64(0.0)),
             ('log_p__', np.float64(0.0)),
             ('log_g__', np.float64(0.0)),
             ('mu[1]', np.float64(-0.009621)),
             ('mu[2]', np.float64(0.0106908))])

See the [API documentation](https://mc-stan.org/cmdstanpy/api.html#cmdstanpy.CmdStanModel.variational) for a full description of all arguments.